In [ ]:
import requests
from bs4 import BeautifulSoup
from getpass import getpass

cas_login_url = "https://cas.finki.ukim.mk/cas/login?service=https%3A%2F%2Fcourses.finki.ukim.mk%2Flogin%2Findex.php"

session = requests.Session()

response = session.get(cas_login_url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')

lt_value = soup.find('input', {'name': 'lt'}).get('value')
execution_value = soup.find('input', {'name': 'execution'}).get('value')

username = '206021'
password = getpass('Enter your password: ')

data = {
    'username': username,
    'password': password,
    'lt': lt_value,
    'execution': execution_value,
    '_eventId': 'submit',
    'submit': 'LOGIN'
}

login_response = session.post(cas_login_url, data=data)

if "Log Out" in login_response.text:
    print("Login successful!")
else:
    print("Login failed!")


Enter your password: ··········
Login failed!


In [ ]:
link = "https://courses.finki.ukim.mk/user/index.php?id=1663&page=0&perpage=7000"

In [ ]:
target_response = session.get(link)
soup = BeautifulSoup(target_response.text, 'html.parser')
#tr_elements are all of the students and prodessors
tr_elements = soup.select('tr:not(.emptyrow)')

In [ ]:
len(tr_elements)

6123

In [ ]:
!pip install tqdm

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from tqdm import tqdm

d1= {}
for tr in tqdm(tr_elements[1:], desc="Processing rows", unit=" row"):
  tmp = tr.find('a')
  link1 = tmp.get("href")
  name = tmp.text
  tmp1 = tr.find(class_="cell c1")
  role = tmp1.text
  target_response = session.get(link1)
  soup = BeautifulSoup(target_response.text, 'html.parser')
  view_more_link = soup.find('a', title='View more')
  if view_more_link != None:
    link2 = view_more_link.get("href")
    target_response = session.get(link2)
    soup = BeautifulSoup(target_response.text, 'html.parser')
  desired_elements = soup.find_all('section', class_='node_category card d-inline-block w-100 mb-3')


  if len(desired_elements)>0:
    email = soup.find('a', href=lambda href: href and 'mailto' in href)

    if email == None:
      email = "Нема"
    else:
      email = email.text





    if len(desired_elements) == 3:
      d  =desired_elements[0]
    elif len(desired_elements) <= 4:
      d  =desired_elements[1]
    else:
      d  =desired_elements[2]
    ul_elements = d.find_all('ul')
    a = ul_elements[1]
    li_elements = a.find_all('li')
    li1=[]
    for li in li_elements:
            li1.append(li.get_text(strip=True))


    d1[(name, email,role)] = li1


Processing rows:   0%|          | 2/6122 [00:04<3:59:07,  2.34s/ row]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.DataFrame()

name = []
role = []
courses = []
email= []
for n,e,r in d1:
  name.append(n)
  role.append(r)
  courses.append("@".join(d1[(n,e,r)]))
  email.append(e)

In [ ]:
df["Names"] = name
df["Emails"]= email
df["Roles"] = role
df["Courses"] = courses

In [ ]:
len(df)

6096

In [ ]:
import os

os.chdir('/content/gdrive/My Drive')

df.to_csv('your_dataframe3.csv', index=False)